In [2]:
# 필요한 라이브러리 임포트
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


# 0번 GPU를 사용한다, 이 서버 같은 경우에는 0,1,2,3 번 GPU가 있다
torch.cuda.set_device(0)

# 데이터셋 로딩 및 준비
batch_size = 64

# 훈련 데이터 다운로드
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True,
)

# 테스트 데이터 다운로드
test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
)

train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# CUDA가 사용가능한 경우 모델을 GPU로 이동
if torch.cuda.is_available():
    model.cuda()

# 훈련 함수
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # CUDA가 사용가능한 경우 데이터를 GPU로 이동
        if torch.cuda.is_available():
            X, y = X.cuda(), y.cuda()

        # 예측 및 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 테스트 함수
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # CUDA가 사용가능한 경우 데이터를 GPU로 이동
            if torch.cuda.is_available():
                X, y = X.cuda(), y.cuda()
                
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    print(f"테스트 오류: \n 정확도: {(accuracy*100):>0.1f}%, 평균 손실: {test_loss:>8f} \n")

# 훈련 실행
learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"에폭 {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("완료!")

에폭 1
-------------------------------
loss: 2.310595  [    0/60000]
loss: 2.309814  [ 6400/60000]
loss: 2.294436  [12800/60000]
loss: 2.304044  [19200/60000]
loss: 2.290190  [25600/60000]
loss: 2.280542  [32000/60000]
loss: 2.269541  [38400/60000]
loss: 2.275205  [44800/60000]
loss: 2.269873  [51200/60000]
loss: 2.256222  [57600/60000]


KeyboardInterrupt: 